In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
import os 

# changing the "current working directory"
os.chdir('/home/ec2-user/SageMaker/')

# confirming cwd is updated 
os.getcwd()

'/home/ec2-user/SageMaker'

In [3]:
dbSP500 = sqlite3.connect('resources/phase3/sp500/sp500.sqlite')

In [48]:
"""select * from dbSP500"""

'select * from dbSP500'

In [4]:
queryTables = """
select name from sqlite_master
where type = 'table'
"""

dfTables = pd.read_sql_query(queryTables,dbSP500)

print(dfTables)

    name
0  sp500


In [5]:
queryRows = """
select count(*) from sp500
"""

dfRows = pd.read_sql_query(queryRows,dbSP500)

print(dfRows)

   count(*)
0     17378


In [6]:
queryData = """
select * from sp500
"""

dfSP500 = pd.read_sql_query(queryData,dbSP500)

dfSP500.head(5) # show the first five records

,id,Date,Open,High,Low,Close,Adj Close,Volume
0,1,1950-01-03,16.66,16.66,16.66,16.66,16.66,1260000
1,2,1950-01-04,16.85,16.85,16.85,16.85,16.85,1890000
2,3,1950-01-05,16.93,16.93,16.93,16.93,16.93,2550000
3,4,1950-01-06,16.98,16.98,16.98,16.98,16.98,2010000
4,5,1950-01-09,17.08,17.08,17.08,17.08,17.08,2520000


In [7]:
i = 0 # set an iteration variable to zero

for col in dfSP500: # set a loop based on the columns in the dataframe 
    print(col) # print the column to screen
    i += 1 # increment i by 1
    
print("\nFound %s columns" % i) # print the column count to screen

id
Date
Open
High
Low
Close
Adj Close
Volume

Found 8 columns


In [8]:
# SQL
queryHighPrice = """
select max(high) from sp500
"""

resultHighPriceSQL = pd.read_sql_query(queryHighPrice, dbSP500)

print("SQL Result: \n\n %s" % resultHighPriceSQL)

print("\n\n\n") # print three blank lines

# Pandas
resultHighPricePandas = dfSP500['High'].max()
# NOTE: The source we hit here is the dataframe with all the records

print("Pandas Result: \n\n %s" % resultHighPricePandas)

SQL Result: 

   max(high)
0    999.61




Pandas Result: 

 999.61


In [9]:
# SQL
queryHighPriceWhen = """
select * from sp500
where high = (select max(high) from sp500)
"""

resultHighPriceWhenSQL = pd.read_sql_query(queryHighPriceWhen, dbSP500)

print("SQL Result: \n\n %s" % resultHighPriceWhenSQL)

print("\n\n\n") # print three blank lines

# pandas

resultHighPriceWhenPandas = dfSP500[dfSP500['High']==dfSP500['High'].max()]

print("Pandas Result: \n\n%s" % resultHighPriceWhenPandas)

SQL Result: 

       id        Date    Open    High     Low   Close Adj Close      Volume
0  15004  2009-08-19  986.88  999.61  980.62  996.46    996.46  4257000000




Pandas Result: 

          id        Date    Open    High     Low   Close Adj Close      Volume
15003  15004  2009-08-19  986.88  999.61  980.62  996.46    996.46  4257000000


In [11]:
# SQL
queryYearlySummaryQuery = """
select
    substr(date, 1,4) as year,
    max(close) as max_close,
    avg(close) as avg_close,
    min(close) as min_close
from sp500
group by substr(date, 1,4)
order by substr(date, 1,4) desc
limit 20
"""

resultYearlySummarySQL = pd.read_sql_query(queryYearlySummaryQuery, dbSP500)

print("SQL Result: \n\n %s" % resultYearlySummarySQL)

SQL Result: 

     year max_close    avg_close min_close
0   2019   2670.71  2593.305294   2447.89
1   2018   2930.75  2746.214422    2351.1
2   2017   2690.16  2449.076375   2257.83
3   2016   2271.72  2094.651270   1829.08
4   2015   2130.82  2061.067738   1867.61
5   2014   2090.57  1931.376111   1741.89
6   2013   1848.36  1643.798968   1457.15
7   2012   1465.77  1379.354160   1277.06
8   2011   1363.61  1267.638810   1099.23
9   2010   1259.78  1139.965516   1022.58
10  2009    998.04   948.046389   1002.63
11  2008    998.01  1220.042055   1003.35
12  2007   1565.15  1477.184343   1374.12
13  2006   1427.09  1310.461235   1223.69
14  2005   1272.74  1207.229444    1137.5
15  2004   1213.55  1130.649444   1063.23
16  2003    999.74   965.227540    1000.3
17  2002    992.72   993.934802   1006.29
18  2001    984.54  1194.178992   1003.45
19  2000   1527.46  1427.221071   1264.74


In [13]:
# Pandas

dfSP500Work = dfSP500

dfSP500Work['Year'] = dfSP500Work['Date'].str.slice(stop=4) # create year column
dfSP500Work['Close'] = pd.to_numeric(dfSP500Work['Close']) # make Close a numeric field

dfSP500Work.groupby(['Year']).agg(
    {'Close': ['max', 'mean', 'min']}
    ).sort_values(by='Year', ascending=False
             ).head(20)

Close                      
          max         mean      min
Year                               
2019  2670.71  2593.305294  2447.89
2018  2930.75  2746.214422  2351.10
2017  2690.16  2449.076375  2257.83
2016  2271.72  2094.651270  1829.08
2015  2130.82  2061.067738  1867.61
2014  2090.57  1931.376111  1741.89
2013  1848.36  1643.798968  1457.15
2012  1465.77  1379.354160  1277.06
2011  1363.61  1267.638810  1099.23
2010  1259.78  1139.965516  1022.58
2009  1127.78   948.046389   676.53
2008  1447.16  1220.042055   752.44
2007  1565.15  1477.184343  1374.12
2006  1427.09  1310.461235  1223.69
2005  1272.74  1207.229444  1137.50
2004  1213.55  1130.649444  1063.23
2003  1111.92   965.227540   800.73
2002  1172.51   993.934802   776.76
2001  1373.73  1194.178992   965.80
2000  1527.46  1427.221071  1264.74

In [49]:
# overall record count
queryRecountCount = """
select 
  count(*) record_count 
from 
  sp500
"""

dfQueryRecordCount = pd.read_sql_query(queryRecountCount, dbSP500)

print(dfQueryRecordCount.to_string(index=False))

 record_count
        17378


In [57]:
# overall record count with dates
queryDateRangesAll = """
select 
  count(*) as total_records,
  min(date) as earliest_date,
  max(date) as latest_date
from sp500
"""

dfQueryDateRangesAll = pd.read_sql_query(queryDateRangesAll, dbSP500)

print(dfQueryDateRangesAll.to_string(index=False))

 total_records earliest_date latest_date
         17378    1950-01-03  2019-01-25


In [63]:
# date range filter: greater/less than
queryDateRangesSome = """
select 
  count(*) as total_records,
  min(date) as earliest_date,
  max(date) as latest_date
from sp500
  where date > '2007-01-01'
    and date < '2009-12-31'
"""

dfQueryDateRangesSome = pd.read_sql_query(queryDateRangesSome, dbSP500)

print(dfQueryDateRangesSome.to_string(index=False))

 total_records earliest_date latest_date
           755    2007-01-03  2009-12-30


In [60]:
# date range filter: BETWEEN
queryDateRangesConditional = """
select 
  count(*) as total_records,
  min(date) as earliest_date,
  max(date) as latest_date
from sp500
  where date between '2007-01-01' and '2009-12-31'
"""

dfQueryDateRangesConditional = pd.read_sql_query(queryDateRangesConditional, dbSP500)

print(dfQueryDateRangesConditional.to_string(index=False))

 total_records earliest_date latest_date
           756    2007-01-03  2009-12-31


In [61]:
# conditionally label date ranges
queryDateRangesConditional = """
select 
  case
    when date < '2007-01-01'
      then 'before'
    when date > '2009-12-31'
      then 'after'
    else 'during'
  end as interested_range,
  count(*) as total_records,
  min(date) as earliest_date,
  max(date) as latest_date
from sp500
  group by 1
"""

dfDateRangesConditional = pd.read_sql_query(queryDateRangesConditional, dbSP500)

print(dfDateRangesConditional.to_string(index=False))

interested_range  total_records earliest_date latest_date
           after           2281    2010-01-04  2019-01-25
          before          14341    1950-01-03  2006-12-29
          during            756    2007-01-03  2009-12-31


In [126]:
# Question 1 - When did the biggest loss occur in the SP 500 during the Great Recession? 
queryRecessionBiggestDropDates = """
select 
  min(date) as Date_Before_Crash,
  max(date) as Date_After_Crash
from sp500
  where date between '2008-09-14' and '2008-10-05'
"""

dfQueryRecessionBiggestDropDates = pd.read_sql_query(queryRecessionBiggestDropDates, dbSP500)

print(dfQueryRecessionBiggestDropDates.to_string(index=False))

Date_Before_Crash Date_After_Crash
       2008-09-15       2008-10-03


In [130]:
# Question 2 - How far did the SP 500 drop during this time frame?

queryRecessionBiggestDropAmount = """
select 
    min(close) as Low_point,
    max(close) as High_point
from sp500
   where date > '2008-09-15'
    and date < '2008-10-05'
"""

dfQueryRecessionBiggestDropAmount = pd.read_sql_query(queryRecessionBiggestDropAmount, dbSP500)

print(dfQueryRecessionBiggestDropAmount.to_string(index=False))

Low_point High_point
  1099.23    1255.08


In [131]:
# Question 3 - What was the Average of the SP 500 during this time frame?

queryRecessionBiggestDropAverage = """
select 
    avg(close) as Average_Point
from sp500
   where date > '2008-09-15'
    and date < '2008-10-05'
"""

dfQueryRecessionBiggestDropAverage = pd.read_sql_query(queryRecessionBiggestDropAverage, dbSP500)

print(dfQueryRecessionBiggestDropAverage.to_string(index=False))

 Average_Point
   1177.325714


In [154]:
# Question 4 - What was the dates ranges and average close for the SP 500 before, during, and after the Great Recession?

queryRecessionBeforeAfterAverages = """
select 
  case
    when date < '2007-12-31'
      then 'Before'
    when date > '2009-06-31'
      then 'After'
    else 'During Recession'
  end as Time_Range,
  avg(close) as Average_Close,
  min(date) as Earliest_Date,
  max(date) as Latest_Date
from sp500
  group by Time_Range
"""

dfQueryRecessionBeforeAfterAverages = pd.read_sql_query(queryRecessionBeforeAfterAverages, dbSP500)

print(dfQueryRecessionBeforeAfterAverages.to_string(index=False))

       Time_Range  Average_Close Earliest_Date Latest_Date
            After    1818.700913    2009-07-01  2019-01-25
           Before     351.019324    1950-01-03  2007-12-28
 During Recession    1099.578069    2007-12-31  2009-06-30


In [149]:
# Question 5 - What was the Difference between the lowest close of the Great Recession and the Highest close of the latest SP 500 closes?

queryRecessionLowAfterHigh = """
select 
    case
    when date between '2008-10-05' and '2009-03-15'
        then 'During Recession'
    when date > '2009-03-15'
        then 'After'
  end as Time_Range,
  avg(close) as Average_Close
from sp500
  group by Time_Range
"""
# delete from queryRecessionLowAfterHigh where 'Time_Range' = "None" - code did not work to get ride of the "None" row

dfQueryRecessionLowAfterHigh = pd.read_sql_query(queryRecessionLowAfterHigh, dbSP500)

print(dfQueryRecessionLowAfterHigh.to_string(index=False))

       Time_Range  Average_Close
             None     363.695209
            After    1790.273933
 During Recession     860.941273
